In [17]:
from atus import loader
import pandas as pd
import re
from matplotlib import pyplot as plt
import seaborn
%matplotlib inline

In [2]:
summary = loader.csv_opener('atussum_2013')

Pertinent columns
=========

##In the Summary


+ TUFINLWGT - statistical weight of respondent
+ TRYHHCHILD - age of youngest child in household
+ TEAGE - age of respondent
+ TESEX - sex of respondent
+ TELFS - working status of respondent, 1 or 2 => employed
+ TRCHILDNUM - number of children in household < 18 yrs
+ TRERNWA - Weekly Earnings/main job
+ TEHRULST - (edited) hours worked per week
+ TRTALONE - non-work related time spend alone
+ TRTFAMILY - non-work time spend with family
+ TRTFRIEND - ""  with friends
+ t020602 - playing with/exercising/walking pets

In [3]:
summary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11385 entries, 0 to 11384
Columns: 413 entries, tucaseid to t500107
dtypes: float64(1), int64(412)
memory usage: 36.0 MB


In [4]:
summary['TELFS'].value_counts()

1    6529
5    3965
4     548
2     295
3      48
dtype: int64

set some basic categories

In [5]:
no_children = summary.TRCHILDNUM == 0
adults = summary.TEAGE >= 18
working = (summary.TELFS == 1) | (summary.TELFS == 2)

## Adults with no children who work

In [6]:
childless_working_adults = summary[no_children & adults & working]
childless_working_adults.set_index('tucaseid', inplace=True)

In [7]:
%%capture
les = [item for item in childless_working_adults.columns if re.match(r't12[0-9]{4}|t13[0-9]{4}|t14[0-9]{4}|t160101|t160102|t160299', item)]

total_weights = childless_working_adults['TUFINLWGT'].sum()

leisure = childless_working_adults[les]

leisure['total'] = leisure.sum(1)

leisure['total'] = leisure['total'].mul(childless_working_adults['TUFINLWGT'], axis=0)


Atus summary comparison:

Atus says childless working adults spend 4.5 hours on average
on leisure activities.

My findings (including religious/spiritual activities (t14), sporting events (t13), telephone calls with family/friends (t16), social/relaxing/leisure (t12) :

In [8]:
print(round(leisure['total'].sum()/(total_weights*60), 1), 'hours')


4.5 hours


In [15]:
%%capture
wait = [item for item in childless_working_adults.columns if re.match(r't1102[0-9]{2}', item)]
waiting_to_eat = childless_working_adults[wait]
waiting_to_eat['total'] = waiting_to_eat.sum(1)
waiting_to_eat['total'] = waiting_to_eat['total'].mul(childless_working_adults['TUFINLWGT'], axis=0)

In [16]:
print(round(waiting_to_eat['total'].sum()/(total_weights), 3), 'minutes')

0.16 minutes
